In [74]:
from IPython.display import display
from numpy.random import RandomState
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer, precision_recall_fscore_support, roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from IPython.display import display
from numpy.random import RandomState
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer, precision_recall_fscore_support, roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as metrics
from collections import Counter
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier

# PREPROCESSING

In [75]:
rs = RandomState(121)

In [76]:
X = pd.read_csv("Dataset.txt", header=None, delimiter=r"\s+",)

In [77]:
X.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

In [78]:
X.isnull().values.any()

True

In [79]:
print(X.head())

   Age  WorkClass  fnlwgt     Education  EducationNum       MaritalStatus  \
0   25    Private  226802          11th             7       Never-married   
1   38    Private   89814       HS-grad             9  Married-civ-spouse   
2   28  Local-gov  336951    Assoc-acdm            12  Married-civ-spouse   
3   44    Private  160323  Some-college            10  Married-civ-spouse   
4   18          ?  103497  Some-college            10       Never-married   

          Occupation Relationship   Race  Gender  CapitalGain  CapitalLoss  \
0  Machine-op-inspct    Own-child  Black    Male          0.0          0.0   
1    Farming-fishing      Husband  White    Male          0.0          0.0   
2    Protective-serv      Husband  White    Male          0.0          0.0   
3  Machine-op-inspct      Husband  Black    Male       7688.0          0.0   
4                  ?    Own-child  White  Female          0.0          0.0   

   HoursPerWeek  NativeCountry Income  
0          40.0  United-Stat

In [80]:
y = X["Income"]
X.drop("Income", axis=1, inplace=True,)

In [81]:
X.Age = X.Age.astype(float)
X.fnlwgt = X.fnlwgt.astype(float)
X.EducationNum = X.EducationNum.astype(float)
X.HoursPerWeek = X.HoursPerWeek.astype(float)

In [82]:
y.shape

(8530,)

In [83]:
X.shape

(8530, 14)

In [84]:
X = X.replace('?', np.NaN)

In [85]:
NAs = pd.concat([X.isnull().sum()], axis=1, keys=['Train'])
NAs[NAs.sum(axis=1) > 0]

,Train
WorkClass,515
MaritalStatus,1
Occupation,516
Relationship,1
Race,1
Gender,1
CapitalGain,1
CapitalLoss,1
HoursPerWeek,1
NativeCountry,138


In [86]:
num_attributes = X.select_dtypes(include='float64')

In [87]:
for i in num_attributes:
    X[i] = X[i].fillna(X[i].mean())

In [88]:
cat_attributes = X.select_dtypes(include=['object'])

In [89]:
X = X.drop('EducationNum', axis=1)

In [90]:
for i in cat_attributes:
    X[i] = X[i].fillna(X[i].mode()[0])

In [91]:
y = y.fillna(y.mode()[0])

In [92]:
for col in X.dtypes[X.dtypes == 'object'].index:
    for_dummy = X.pop(col)
    X = pd.concat([X, pd.get_dummies(for_dummy, prefix=col)], axis=1)

In [93]:
y

0       <=50K
1       <=50K
2        >50K
3        >50K
4       <=50K
        ...  
8525    <=50K
8526     >50K
8527    <=50K
8528    <=50K
8529    <=50K
Name: Income, Length: 8530, dtype: object

In [94]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state= 40)

In [95]:
kfold = StratifiedKFold(n_splits = 3 , random_state=40)

### DECISION TREE 

In [96]:
tree = DecisionTreeClassifier()
model_tree = tree.fit(x_train,y_train)
model_tree

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [97]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [98]:
model_tree = tree.fit(x_train,y_train)
pred_tree = tree.predict(x_test)
a1 = accuracy_score(y_test,pred_tree)
print("The Accuracy of Desicion Tree is ", a1)

The Accuracy of Desicion Tree is  0.7966002344665886


In [99]:
cross_val_score(model_tree,X,y,scoring= "accuracy", cv=kfold).mean()

0.806212600952889

### RANDOM FOREST

In [100]:
rf = RandomForestClassifier()
model_rf = rf.fit(x_train,y_train)
pred_rf = rf.predict(x_test)
a2 = accuracy_score(y_test, pred_rf)
print("The Accuracy of Random Forest is ", a2)

The Accuracy of Random Forest is  0.8335287221570926


In [101]:
cross_val_score(model_rf,X,y,scoring= "accuracy", cv=kfold).mean()

0.8371636787019963

### LOGISTIC REGRESSION

In [102]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()

model_lg = lg.fit(x_train,y_train)
pred_lg = lg.predict(x_test)
a3 = accuracy_score(y_test, pred_lg)
print("The Accuracy of logistic regression is ", a3)

The Accuracy of logistic regression is  0.8001172332942555


In [103]:
cross_val_score(model_lg,X,y,scoring= "accuracy", cv=kfold).mean()

0.8026968389567379

### KNN

In [104]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier()

In [105]:
model_knn =knn.fit(x_train,y_train) 
pred_knn = knn.predict(x_test)
a4 = accuracy_score(y_test, pred_knn)
print("The Accuracy of KNN is ", a4)

The Accuracy of KNN is  0.764947245017585


In [106]:
cross_val_score(model_knn,X,y,scoring= "accuracy", cv=kfold).mean()

0.7747952568679638

### RANDOM FOREST BEST PARAMETERS

In [107]:
rf_param = {
    "n_estimators": [25,50,100],
    "criterion" : ["gini"],
    "max_depth" : [3,4,5,6],
    "max_features" : ["auto","sqrt","log2"],
    "random_state" : [123]
}

In [108]:
#GridSearchCV(rf, rf_param, cv = kfold)

In [109]:
grid =GridSearchCV(rf, rf_param, cv = kfold)

In [110]:
grid.fit(x_train,y_train).best_params_

{'criterion': 'gini',
 'max_depth': 6,
 'max_features': 'auto',
 'n_estimators': 50,
 'random_state': 123}

In [111]:
rf1 = RandomForestClassifier(criterion = 'gini',
    max_depth = 6,
    max_features = 'auto',
    n_estimators = 50,
    random_state = 123)
model_rf1 = rf1.fit(x_train,y_train)
pred_rf1 = rf1.predict(x_test)
accuracy_score(y_test, pred_rf1)

0.8382180539273154

In [112]:
cross_val_score(model_rf1,X,y,scoring= "accuracy", cv=kfold).mean()

0.8379832121024364

### DECISION TREE BEST PARAMETERS

In [113]:
dt_param = {
    'criterion':['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth' : np.arange(1,20),
    'min_samples_leaf' : [1,2,4,5,10]
}

In [114]:
dt_grid =GridSearchCV(tree, dt_param, cv = kfold)

In [115]:
dt_grid.fit(x_train,y_train).best_params_

{'criterion': 'gini',
 'max_depth': 8,
 'min_samples_leaf': 1,
 'splitter': 'best'}

In [116]:
tree2 = DecisionTreeClassifier(criterion= 'gini', 
                               max_depth= 8, min_samples_leaf= 4, splitter= 'best')
model_tree2 = tree2.fit(x_train,y_train)
pred_tree2 = tree2.predict(x_test)
accuracy_score(y_test, pred_tree2)

0.8452520515826495

In [117]:
cross_val_score(model_tree2,X,y,scoring= "accuracy", cv=kfold).mean()

0.8519333432970647

### 5 FOLD CV SCORES

In [118]:
#DECISION TREE
cross_val_score(tree,X,y,scoring= "accuracy", cv=kfold).mean()

0.805978931657051

In [119]:
#RANDOM FOREST
cross_val_score(rf,X,y,scoring= "accuracy", cv=kfold).mean()

0.8391556001613992

In [120]:
#LOGISTIC REGRESSION
cross_val_score(lg,X,y,scoring= "accuracy", cv=kfold).mean()

0.8026968389567379

In [121]:
#KNN
cross_val_score(knn,X,y,scoring= "accuracy", cv=kfold).mean()

0.7747952568679638

In [122]:
#DECISION TREE BEST PARAMETERS
cross_val_score(tree2,X,y,scoring= "accuracy", cv=kfold).mean()

0.8515817258568396

In [123]:
#RANDOM FOREST  BEST PARAMETERS
cross_val_score(rf1,X,y,scoring= "accuracy", cv=kfold).mean()

0.8379832121024364

### ENSEMBLE VOTING CLASSIFIER

In [124]:
from sklearn.ensemble import VotingClassifier

In [125]:
model_vote = VotingClassifier(
    estimators=[('logistic Regression', lg), 
                ('random forrest', rf), 
                ('knn neighbors', knn),
                (' decision tree', tree)], voting='soft')
model_vote = model_vote.fit(x_train, y_train)

In [126]:
vote_pred = model_vote.predict(x_test)

In [127]:
a5 =  accuracy_score(y_test, vote_pred)
print("The Accuracy of voting classifier is ", a5)

The Accuracy of voting classifier is  0.8270808909730364


In [128]:
cross_val_score(model_vote,X,y,scoring= "accuracy", cv=kfold).mean()

0.8379832945836562

### RANDOM FOREST Ensemble Bagging 

In [129]:
from sklearn.ensemble import BaggingClassifier

In [130]:
bagg = BaggingClassifier(base_estimator=rf1,n_estimators=15)

In [131]:
model_bagg =bagg.fit(x_train,y_train) 
pred_bagg = bagg.predict(x_test)

In [132]:
a6 = accuracy_score(y_test, pred_bagg)
print("The Accuracy of BAAGING is ", a6)

The Accuracy of BAAGING is  0.8305978898007034


In [133]:
cross_val_score(model_bagg,X,y,scoring= "accuracy", cv=kfold).mean()

0.831770809104475

### RANDOM FOREST ENSEMBLE ADA BOOST 

In [134]:
from sklearn.ensemble import AdaBoostClassifier

In [135]:
Adaboost = AdaBoostClassifier(base_estimator=rf1, n_estimators=15)

In [136]:
model_boost =Adaboost.fit(x_train,y_train) 
pred_boost = Adaboost.predict(x_test)

In [137]:
a7 = accuracy_score(y_test, pred_boost)
print("The Accuracy of BOOSTING is ", a7)

The Accuracy of BOOSTING is  0.8563892145369285


In [138]:
cross_val_score(model_boost,X,y,scoring= "accuracy", cv=kfold).mean()

0.8609620675468402

### DECISION TREE ENSEMBLE BAGGING

In [139]:
dt_bagg = BaggingClassifier(base_estimator=tree2,n_estimators=15)

In [140]:
dt_model_bagg =dt_bagg.fit(x_train,y_train) 
dt_pred_bagg = dt_bagg.predict(x_test)

In [141]:
a8 = accuracy_score(y_test, dt_pred_bagg)
print("The Accuracy of BAAGING is ", a8)

The Accuracy of BAAGING is  0.8464243845252052


In [142]:
cross_val_score(dt_model_bagg,X,y,scoring= "accuracy", cv=kfold).mean()

0.8580301901060148

### DECISION TREE ADA BOOST

In [143]:
dt_Adaboost = AdaBoostClassifier(base_estimator=tree2, n_estimators=15)

In [144]:
dt_model_boost =dt_Adaboost.fit(x_train,y_train) 
dt_pred_boost = dt_Adaboost.predict(x_test)

In [145]:
a9 = accuracy_score(y_test, dt_pred_boost)
print("The Accuracy of BOOSTING is ", a9)

The Accuracy of BOOSTING is  0.8212192262602579


In [146]:
cross_val_score(dt_model_boost,X,y,scoring= "accuracy", cv=kfold).mean()

0.8220396747864479